# ITU 5G Energy Consumption Challenge - GitData

<font color="red">Please make a copy of this Jupyter Notebook to your own Google Drive or local machine before changing anything.</font>

This Jupyter Notebook will also be updated regularly on GitHub: https://github.com/GitData-GA/itu5g/tree/main/code/ITU_5G_Energy_Consumption_Challenge.ipynb

In [ ]:
# Import all libraries here
import random
random.seed(88)
import pandas as pd
import numpy as np
np.random.seed(88)
import tensorflow as tf
tf.random.set_seed(88)
from tensorflow import keras
keras.utils.set_random_seed(88)
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# Output format
# Description: The file contains days, hours, and base stations for which we
# need energy consumption estimation and will be used for scoring predictions.
power_pred = pd.read_csv('https://data.gd.edu.kg/bwz7t527/power_consumption_prediction.csv')
# Dataset 1
# Description: Energy consumption data (ECdata.csv): include hour-level energy
# consumption specifications (e.g., total energy consumption of the base stations).
ECdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/ECdata.csv')
# Dataset 2
# Description: Cell-level data (CLdata.csv): include hour-level counters,
# including service compliance counters (e.g., load) and energy-saving methods
# counters (e.g., duration of energy saving mode activation).
CLdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/CLdata.csv')
# Dataset 3
# Description: Base Station basic information (BSinfo.csv): include
# configuration parameters and hardware attributes
BSinfo = pd.read_csv('https://data.gd.edu.kg/bwz7t527/BSinfo.csv')

In [ ]:
data = ECdata.merge(CLdata, on=["Time", "BS"], how="inner")
data = data.merge(BSinfo, on="BS", how="inner")
data = data.sort_values(by='Time', ascending=True)
# Convert date strings to datetime objects
data['Time'] = pd.to_datetime(data['Time'])

# Sort the dataset by date
data.sort_values(by='Time', inplace=True)

# Split Year, Month, Day, and Hour
data['Year'] = data['Time'].dt.year
data['Month'] = data['Time'].dt.month
data['Day'] = data['Time'].dt.day
data['Hour'] = data['Time'].dt.hour

# Extract target variable 'Energy'
y = data['Energy']

# Drop unnecessary columns (e.g., 'Time', 'Energy')
data.drop(columns=['Time', 'Energy'], inplace=True)

# One-hot encode categorical features
categorical_cols = ['BS', 'CellName_x', 'CellName_y', 'RUType', 'Mode', ]
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
data_encoded[data_encoded.columns] = scaler.fit_transform(data_encoded[data_encoded.columns])

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data_encoded, y, test_size=0.2, shuffle=False)

# Convert data to numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
# Time series transformation
X_train_ts = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_ts = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
# LSTM Model - Time series
lstm_model = keras.Sequential([
    layers.LSTM(units=128, recurrent_dropout=0.15),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
lstm_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = lstm_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = lstm_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

lstm_model.summary()

Epoch 1/50
2514/2514 [==============================] - 46s 15ms/step - loss: 26.4170 - mean_absolute_percentage_error: 26.4170 - val_loss: 8.5656 - val_mean_absolute_percentage_error: 8.5656
Epoch 2/50
2514/2514 [==============================] - 36s 14ms/step - loss: 14.1315 - mean_absolute_percentage_error: 14.1315 - val_loss: 7.5778 - val_mean_absolute_percentage_error: 7.5778
Epoch 3/50
2514/2514 [==============================] - 36s 14ms/step - loss: 12.5573 - mean_absolute_percentage_error: 12.5573 - val_loss: 7.4844 - val_mean_absolute_percentage_error: 7.4844
Epoch 4/50
2514/2514 [==============================] - 41s 16ms/step - loss: 11.6290 - mean_absolute_percentage_error: 11.6290 - val_loss: 7.0232 - val_mean_absolute_percentage_error: 7.0232
Epoch 5/50
2514/2514 [==============================] - 35s 14ms/step - loss: 10.8630 - mean_absolute_percentage_error: 10.8630 - val_loss: 6.4399 - val_mean_absolute_percentage_error: 6.4399
Epoch 6/50
2514/2514 [==================

In [ ]:
# GRU Model - Time series
gru_model = keras.Sequential([
    layers.GRU(units=128, recurrent_dropout=0.15),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
gru_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = gru_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = gru_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

gru_model.summary()

Epoch 1/50
2514/2514 [==============================] - 39s 13ms/step - loss: 25.4505 - mean_absolute_percentage_error: 25.4505 - val_loss: 8.6548 - val_mean_absolute_percentage_error: 8.6548
Epoch 2/50
2514/2514 [==============================] - 34s 14ms/step - loss: 14.3029 - mean_absolute_percentage_error: 14.3029 - val_loss: 7.8732 - val_mean_absolute_percentage_error: 7.8732
Epoch 3/50
2514/2514 [==============================] - 35s 14ms/step - loss: 12.6314 - mean_absolute_percentage_error: 12.6314 - val_loss: 7.6799 - val_mean_absolute_percentage_error: 7.6799
Epoch 4/50
2514/2514 [==============================] - 34s 14ms/step - loss: 11.6773 - mean_absolute_percentage_error: 11.6773 - val_loss: 7.0883 - val_mean_absolute_percentage_error: 7.0883
Epoch 5/50
2514/2514 [==============================] - 32s 13ms/step - loss: 10.6589 - mean_absolute_percentage_error: 10.6589 - val_loss: 6.8378 - val_mean_absolute_percentage_error: 6.8378
Epoch 6/50
2514/2514 [==================

In [ ]:
# RNN Model - Time series
rnn_model = keras.Sequential([
    layers.GRU(units=128, recurrent_dropout=0.15),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
rnn_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = rnn_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = rnn_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

rnn_model.summary()

Epoch 1/50
2514/2514 [==============================] - 43s 15ms/step - loss: 26.9694 - mean_absolute_percentage_error: 26.9694 - val_loss: 8.7289 - val_mean_absolute_percentage_error: 8.7289
Epoch 2/50
2514/2514 [==============================] - 35s 14ms/step - loss: 14.0243 - mean_absolute_percentage_error: 14.0243 - val_loss: 7.5591 - val_mean_absolute_percentage_error: 7.5591
Epoch 3/50
2514/2514 [==============================] - 36s 14ms/step - loss: 12.4342 - mean_absolute_percentage_error: 12.4342 - val_loss: 7.0117 - val_mean_absolute_percentage_error: 7.0117
Epoch 4/50
2514/2514 [==============================] - 34s 14ms/step - loss: 11.4387 - mean_absolute_percentage_error: 11.4387 - val_loss: 6.6606 - val_mean_absolute_percentage_error: 6.6606
Epoch 5/50
2514/2514 [==============================] - 34s 14ms/step - loss: 10.6393 - mean_absolute_percentage_error: 10.6393 - val_loss: 6.5629 - val_mean_absolute_percentage_error: 6.5629
Epoch 6/50
2514/2514 [==================

In [ ]:
# Regression Model - Regression
model = keras.Sequential([
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

model.summary()

Epoch 1/50
2514/2514 [==============================] - 22s 7ms/step - loss: 26.6097 - mean_absolute_percentage_error: 26.6097 - val_loss: 8.7298 - val_mean_absolute_percentage_error: 8.7298
Epoch 2/50
2514/2514 [==============================] - 19s 8ms/step - loss: 14.3497 - mean_absolute_percentage_error: 14.3497 - val_loss: 7.6365 - val_mean_absolute_percentage_error: 7.6365
Epoch 3/50
2514/2514 [==============================] - 18s 7ms/step - loss: 12.8393 - mean_absolute_percentage_error: 12.8393 - val_loss: 7.7033 - val_mean_absolute_percentage_error: 7.7033
Epoch 4/50
2514/2514 [==============================] - 19s 8ms/step - loss: 11.9661 - mean_absolute_percentage_error: 11.9661 - val_loss: 6.7577 - val_mean_absolute_percentage_error: 6.7577
Epoch 5/50
2514/2514 [==============================] - 18s 7ms/step - loss: 11.2306 - mean_absolute_percentage_error: 11.2306 - val_loss: 6.8181 - val_mean_absolute_percentage_error: 6.8181
Epoch 6/50
2514/2514 [=======================